In [54]:
import pandas as pd
import numpy as np
import os
from utils.config import Config
test_config = Config()

In [55]:
os.chdir('/mnt/jw01-aruk-home01/projects/psa_functional_genomics/RA_challenge/RA2_alpine_lads')

In [5]:
training_csv = "data/train/training.csv"

In [6]:
info = pd.read_csv(training_csv)
features = info.columns

In [7]:
parts = ["LH","RH","LF","RF"]
dataframes = {}
for part in parts:
    dataframes[part] = info.loc[:,["Patient_ID"]+[s for s in features if part in s]].copy()
    dataframes[part]["total_fig_score"] = dataframes[part].loc[:,[s for s in features if part in s]].sum(axis=1)
    dataframes[part]["Patient_ID"] = dataframes[part]["Patient_ID"].astype(str) + f"-{part}"

In [8]:
# use left as reference
# flip the Rights
dataframes["RH"].columns = dataframes["LH"].columns 
dataframes["RF"].columns = dataframes["LF"].columns 
data_hands = pd.concat((dataframes["RH"],dataframes["LH"]))
data_feet = pd.concat((dataframes["RF"],dataframes["LF"]))

In [9]:
data_feet

Patient_ID  LF_mtp_E__ip  LF_mtp_E__1  LF_mtp_E__2  LF_mtp_E__3  \
0    UAB001-RF             0            0            0            0   
1    UAB002-RF             0            0            0            0   
2    UAB003-RF             0            0            0            0   
3    UAB005-RF             0            0            0            0   
4    UAB006-RF             0            0            0            0   
..         ...           ...          ...          ...          ...   
363  UAB716-LF             0            0            0            0   
364  UAB717-LF             0            0            0            0   
365  UAB719-LF             1            0            0            0   
366  UAB720-LF             0            0            0            0   
367  UAB722-LF             0            1            0            0   

     LF_mtp_E__4  LF_mtp_E__5  LF_mtp_J__ip  LF_mtp_J__1  LF_mtp_J__2  \
0              0            0             0            2            0   
1              0            0             0            0            0   
2              0            0             0            0            0   
3              0            0             0            0            0   
4              0            0             0            0            0   
..           ...          ...           ...          ...          ...   
363            0            0             0            0            0   
364            0            0             0            0            0   
365            0            0             0            0            0   
366            0            0             0            0            0   
367            0            0             0            0            0   

     LF_mtp_J__3  LF_mtp_J__4  LF_mtp_J__5  total_fig_score  
0              0            0            2                4  
1              0            0            0                0  
2              0            0            0                0  
3              0            0            0                0  
4              0            0            0                0  
..           ...          ...          ...              ...  
363            0            0            0                0  
364            0            0            0                0  
365            0            0            0                1  
366            0            0            0                0  
367            0            0            3                4  

[736 rows x 14 columns]

In [10]:
import tensorflow as tf
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow import keras 

In [38]:
test_config.train_location+"/"+data_feet["Patient_ID"].astype(str)+".jpg"

0      data/train/UAB001-RF.jpg
1      data/train/UAB002-RF.jpg
2      data/train/UAB003-RF.jpg
3      data/train/UAB005-RF.jpg
4      data/train/UAB006-RF.jpg
                 ...           
363    data/train/UAB716-LF.jpg
364    data/train/UAB717-LF.jpg
365    data/train/UAB719-LF.jpg
366    data/train/UAB720-LF.jpg
367    data/train/UAB722-LF.jpg
Name: Patient_ID, Length: 736, dtype: object

In [47]:
dataset = tf.data.Dataset.from_tensor_slices((data_feet["Patient_ID"].values, data_feet.loc[:, data_feet.columns != 'Patient_ID'].values))

In [48]:
def load_images(file,y):
    path = test_config.train_location+"/"+file+".jpg"
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=1)
    img = tf.image.convert_image_dtype(img, tf.float32)
#     tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])
    return img, y

In [49]:
dataset = dataset.map(load_images)

In [53]:
for image, label in dataset.take(1):
    print(image)
    print(label)
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())

tf.Tensor(
[[[0.12941177]
  [0.12941177]
  [0.12941177]
  ...
  [0.14509805]
  [0.14509805]
  [0.14509805]]

 [[0.13333334]
  [0.13333334]
  [0.13333334]
  ...
  [0.14117648]
  [0.14117648]
  [0.14117648]]

 [[0.13333334]
  [0.13333334]
  [0.13333334]
  ...
  [0.14117648]
  [0.14117648]
  [0.14117648]]

 ...

 [[0.18823531]
  [0.18823531]
  [0.18823531]
  ...
  [0.20784315]
  [0.20392159]
  [0.20000002]]

 [[0.19215688]
  [0.19215688]
  [0.19215688]
  ...
  [0.21176472]
  [0.20392159]
  [0.19607845]]

 [[0.19215688]
  [0.19215688]
  [0.19215688]
  ...
  [0.21176472]
  [0.20392159]
  [0.19607845]]], shape=(1506, 1047, 1), dtype=float32)
tf.Tensor([0 0 0 0 0 0 0 2 0 0 0 2 4], shape=(13,), dtype=int64)
Image shape:  (1506, 1047, 1)
Label:  [0 0 0 0 0 0 0 2 0 0 0 2 4]


In [31]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [59]:
image_path = Config.train_location + "/UAB070-LH.jpg"
d = Image.open(str(image_path))


In [62]:
np.asarray(d).shape


(1670, 984)

In [56]:
for sample in dataframes["LH"]["Patient_ID"]:
    image_path = Config.train_location + "/" + sample +"-LH.jpg"
    d = Image.open(str(image_path))
    print(sample,np.asarray(d)[:,:,0].shape)

UAB001 (1609, 1253)
UAB002 (1859, 1407)
UAB003 (1729, 1147)
UAB005 (2163, 1591)
UAB006 (1836, 1553)
UAB007 (2097, 1515)
UAB008 (2007, 1475)
UAB009 (1575, 1129)
UAB012 (1599, 1414)
UAB013 (2083, 1617)
UAB018 (2091, 1546)
UAB021 (1838, 1723)
UAB022 (479, 364)
UAB023 (1549, 1298)
UAB024 (2106, 1750)
UAB028 (1039, 756)
UAB030 (1553, 1285)
UAB031 (1515, 1083)
UAB032 (1654, 1270)
UAB033 (1619, 1133)
UAB035 (1730, 1545)
UAB036 (1446, 1071)
UAB037 (1663, 1202)
UAB039 (1757, 1259)
UAB040 (1570, 1523)
UAB041 (1580, 1314)
UAB042 (1579, 1255)
UAB045 (1809, 1442)
UAB046 (1643, 1271)
UAB049 (1638, 1198)
UAB050 (1638, 1512)
UAB051 (1442, 1164)
UAB052 (565, 434)
UAB054 (472, 342)
UAB055 (2149, 1458)
UAB056 (2510, 2000)
UAB057 (2225, 1588)
UAB058 (2510, 2000)
UAB059 (2325, 1511)
UAB060 (1380, 795)
UAB061 (2038, 1454)
UAB062 (1610, 1106)
UAB063 (1715, 1259)
UAB064 (1134, 705)
UAB065 (1605, 1174)
UAB066 (2054, 1544)
UAB067 (1201, 967)
UAB068 (1850, 1621)
UAB069 (1221, 863)


IndexError: too many indices for array